# Data preprocessing

## 1. Import and view data

In [ ]:
import pandas as pd

In [ ]:
speeches_raw = pd.read_csv("../data/interim/bundestag_speeches_processed.csv")

In [ ]:
speeches_raw.head()

In [ ]:
speeches_raw.info()

In [ ]:
tweets_raw = pd.read_csv("../data/interim/Bundestag_Tweets.csv", index_col = "Unnamed: 0")

In [ ]:
tweets_raw.head()

In [ ]:
tweets_raw.info()

## 2. Speeches

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

import spacy
from spacy.language import Language

In [ ]:
# Create spacy pipeline
pipeline_exclude = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner', 'morphologizer']
nlp_speeches = spacy.load('de_core_news_sm', exclude=pipeline_exclude)
nlp_speeches.Defaults.stop_words |= {"\n  ","\n\n  "}

@Language.component("Lemmatize text")
def lemmatize_text(doc):
    doc = [token.lemma_ for token in doc]
    doc = ' '.join(doc)
    return nlp_speeches.make_doc(doc)

@Language.component("Lowercase Text")
def lowercase(doc):
    doc = [token.lower_ for token in doc]
    doc = ' '.join(doc)
    return nlp_speeches.make_doc(doc)

@Language.component("Remove stopwords and punctuation")
def remove_stopwords(doc):
    doc = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp_speeches.add_pipe("Lemmatize text", name="Lemmatize text", last=True)
nlp_speeches.add_pipe("Lowercase Text", name="Lowercase Text", last=True)
nlp_speeches.add_pipe("Remove stopwords and punctuation", name="Remove stopwords and punctuation", last=True)

In [ ]:
speeches = speeches_raw.speech_content.dropna()[0:100]
test_speeches = speeches.progress_apply(nlp_speeches)

In [ ]:
test_speeches[0]

## 3. Twitter data

In [ ]:
# Create emoji matcher
import re
emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

In [ ]:
# Create spacy pipeline
pipeline_exclude = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner', 'morphologizer']
nlp_twitter = spacy.load('de_core_news_sm', exclude=pipeline_exclude)
nlp_twitter.Defaults.stop_words |= {"\n    "}

@Language.component("Lemmatize text")
def lemmatize_text(doc):
    doc = [token.lemma_ for token in doc]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Lowercase Text")
def lowercase(doc):
    doc = [token.lower_ for token in doc]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Remove URLs")
def remove_urls(doc):
    doc = [token.text for token in doc if not token.like_url]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Remove emojis")
def remove_emojis(doc):
    doc = [token.text for token in doc if not re.match(emoji, token.text)]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Remove mentions")
def remove_mentions(doc):
    doc = [token.text for token in doc if not re.match("@.*", token.text)]
    doc = ' '.join(doc)
    return nlp_twitter.make_doc(doc)

@Language.component("Remove stopwords and punctuation")
def remove_stopwords(doc):
    doc = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp_twitter.add_pipe("emoji", first=True)
nlp_twitter.add_pipe("Lemmatize text", name="Lemmatize text", last=True)
nlp_twitter.add_pipe("Lowercase Text", name="Lowercase Text", last=True)
nlp_twitter.add_pipe("Remove URLs", name="Remove URLs", last=True)
nlp_twitter.add_pipe("Remove emojis", name="Remove emojis", last=True)
nlp_twitter.add_pipe("Remove mentions", name="Remove mentions", last=True)
nlp_twitter.add_pipe("Remove stopwords and punctuation", name="Remove stopwords and punctuation", last=True)

In [ ]:
tweets = tweets_raw.Text[0:100]
test_tweets = tweets.progress_apply(nlp_twitter)